In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data_path="E:\\5 Code\\2025_cu_qmim\\data"

In [ ]:
metric_data=os.path.join(data_path,'price_metrics.parquet')
px_all = pd.read_parquet(metric_data)
px_all.head()

In [ ]:
# Extract PX_LAST only (MultiIndex columns)
px = px_all.xs("PX_LAST", axis=1, level="metric")
# Resample to month end
px_m = px.resample("M").last()
px_m

In [ ]:
# Compute 6-1 and 12-1 momentum
mom_6_1 = np.log(px_m.shift(1)) - np.log(px_m.shift(7))

# Winsorize
def winsorize(row, lower=0.01, upper=0.99):
    if row.isna().all():
        return row
    lo, hi = row.quantile([lower, upper])
    return row.clip(lo, hi)

mom6_w = mom_6_1.apply(winsorize, axis=1)


In [ ]:
def stand_data(df,threshold):
    df=df.replace([np.inf, -np.inf], np.nan)
    df=df.fillna(0)
    df=df.apply(lambda x: np.where(x>threshold,threshold,x))
    df=df.apply(lambda x: np.where(x<-threshold,-threshold,x))
    return df

In [ ]:
# Z-score
mom6_z  = mom6_w.sub(mom6_w.mean(axis=1), axis=0).div(mom6_w.std(axis=1), axis=0)


# Baseline portfolio
def build_positions(signal, long_q=0.8, short_q=0.2):
    pos = pd.DataFrame(index=signal.index, columns=signal.columns)

    for dt, row in signal.iterrows():
        r = row.rank(pct=True)
        pos.loc[dt] = (r >= long_q).astype(int) - (r <= short_q).astype(int)

    return pos

pos6_m  = build_positions(mom6_z)


# Compute daily return
daily_ret = px.pct_change()

# In-sample window
px_bt = px.loc["2010-01-01":"2020-12-31"]
daily_ret_bt = daily_ret.loc["2010-01-01":"2020-12-31"]

pos6_d  = pos6_m.reindex(daily_ret_bt.index).ffill()


# Compute strategy returns
strategy6 = (pos6_d.shift(1) * daily_ret_bt).mean(axis=1).fillna(0)


# Cum returns
cum6 = (1 + strategy6).cumprod()


# Plot


plt.figure(figsize=(12,6))
plt.plot(cum6, label="6-1 Momentum")
plt.title("Baseline Momentum Backtest (2010–2020)")
plt.grid(True)
plt.legend()
plt.show()
